In [13]:
import logging
import os

#import wiki_old as w # old wiki
import wiki as w 
 
#from gensim.models import word2vec # for orignal w2v
from localgensim.gensim2.models import word2vec 




#from gensim.models.fasttext import FastText
#from gensim.models.word2vec import Word2Vec # not in use
#from localgensim.gensim2.models.word2vec import Word2Vec # not in use

from tqdm import tqdm

WIKIXML = '/home/manni/data/wiki/enwiki-20211120-pages-articles-multistream.xml.bz2'
#WIKIXML = '/home/manni/data/wiki/enwiki-20211120-pages-articles-multistream1.xml-p1p41242.bz2'

In [2]:
print(w.__file__)
print(word2vec.__file__)

/home/manni/ner-s2s/word_embedding/wiki.py
/home/manni/ner-s2s/word_embedding/localgensim/gensim2/models/word2vec.py


In [3]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [14]:
logging.basicConfig(format='[%(asctime)s] %(message)s', level=logging.INFO)
os.makedirs('data/', exist_ok=True)

# Test corpus

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
sentences = MyCorpus()

In [ ]:
logging.info('Training model %s', 'word2vec')
model = Word2Vec(sentences, window=5, sg=1, hs=0, negative=10, size=300, sample=0, 
                 workers=1, iter=1, min_count=1)

logging.info('Training done.')

In [ ]:
sentences = w.WikiSentences(WIKIXML, 'en')

# Conll corpus

In [ ]:
sentences = sv.load("conll_corpus")

In [ ]:
model = word2vec.Word2Vec(sentences, window=5, sg=1, hs=0, negative=5, size=300, sample=0, workers=1, iter=1, min_count=1)

In [ ]:
#emb_file = '/mnt/nfs/resdata0/manni/wiki/conll_spx2g.txt'
emb_file = '/mnt/nfs/resdata0/manni/wiki/conll_w2v.txt'

In [ ]:
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(model.wv.vocab), 300))
    for word in tqdm(model.wv.vocab):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')

# skip if sentence made

In [5]:
# loc = 'num'|'lr'|'ent'
# pos = True|False
# download latest wiki dump
#w.download_wiki_dump('en', WIKIXML)

# parse wiki dump
wiki_sentences = w.WikiSentences(WIKIXML, 'en',lower=True) # Orignal
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='EM',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='DEP',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='UNS',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='UNSEM',lower=True,pos=False,loc=False)

[2022-09-14 18:06:58,180] Parsing wiki corpus Altered...
[2022-09-14 18:07:07,165] adding document #0 to Dictionary(0 unique tokens: [])
[2022-09-14 18:08:15,143] adding document #10000 to Dictionary(493671 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:09:16,028] adding document #20000 to Dictionary(699785 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:10:07,568] adding document #30000 to Dictionary(854217 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:10:52,879] adding document #40000 to Dictionary(985354 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:13:47,022] adding document #100000 to Dictionary(1329619 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:14:25,475] adding document #110000 to Dictionary(1420571 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 

[2022-09-14 18:24:46,096] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:24:46,158] adding document #280000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:25:16,583] discarding 60294 tokens: [('yastur', 1), ('megaweapon', 1), ('skedars', 1), ('chernen', 1), ('eggenweiler', 1), ('orcthira', 1), ('buouyancy', 1), ('overtiredness', 1), ('scubber', 1), ('liberalhistory', 1)]...
[2022-09-14 18:25:16,584] keeping 2000000 tokens which were in no less than 0 and no more than 290000 (=100.0%) documents
[2022-09-14 18:25:20,928] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:25:20,992] adding document #290000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:25:50,358] discarding 56919 tokens: [('茂才', 

[2022-09-14 18:31:49,455] keeping 2000000 tokens which were in no less than 0 and no more than 410000 (=100.0%) documents
[2022-09-14 18:31:53,799] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:31:53,867] adding document #410000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:32:21,550] discarding 44186 tokens: [('horsgeat', 1), ('hàslam', 1), ('inshaig', 1), ('inverchaolain', 1), ('invercreran', 1), ('kilcalmonell', 1), ('kilmanshenachan', 1), ('kilmorich', 1), ('kinlocheil', 1), ('knipoch', 1)]...
[2022-09-14 18:32:21,552] keeping 2000000 tokens which were in no less than 0 and no more than 420000 (=100.0%) documents
[2022-09-14 18:32:25,028] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:32:25,094] adding document #420000 to Dictionary(2000000 unique 

[2022-09-14 18:38:39,049] discarding 42178 tokens: [('замкавая', 1), ('кавальская', 1), ('карэліцкая', 1), ('мугір', 1), ('мячэтавая', 1), ('міцкевіч', 1), ('наваградак', 1), ('наваградзкая', 1), ('наваградзкі', 1), ('паліклініка', 1)]...
[2022-09-14 18:38:39,050] keeping 2000000 tokens which were in no less than 0 and no more than 540000 (=100.0%) documents
[2022-09-14 18:38:42,185] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:38:42,235] adding document #540000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:39:07,493] discarding 43614 tokens: [('hendratmo', 1), ('jakákoliv', 1), ('kirkaryan', 1), ('meghedin', 1), ('melodiyu', 1), ('ntifi', 1), ('tiškevič', 1), ('ugadaika', 1), ('ugaday', 1), ('ugadayka', 1)]...
[2022-09-14 18:39:07,494] keeping 2000000 tokens which were in no less than 0 and no more than 550000 (=100.0%) documents
[20

[2022-09-14 18:45:04,843] discarding 37686 tokens: [('balmant', 1), ('equidirectional', 1), ('iwanuch', 1), ('kangerdluarssaq', 1), ('litchfieldite', 1), ('rembrandt_man_in_armour', 1), ('rosencreütz', 1), ('check_something', 1), ('oddingseles', 1), ('privman', 1)]...
[2022-09-14 18:45:04,845] keeping 2000000 tokens which were in no less than 0 and no more than 670000 (=100.0%) documents
[2022-09-14 18:45:09,453] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:45:09,527] adding document #670000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 18:45:33,675] discarding 37567 tokens: [('resoul', 1), ('אלקוש', 1), ('hikutavake', 1), ('namukulu', 1), ('ethylphenethylamine', 1), ('etylfenetylamin', 1), ('webtryp', 1), ('tallyband', 1), ('allüberall', 1), ('aufblüh', 1)]...
[2022-09-14 18:45:33,677] keeping 2000000 tokens which were in no less than 0

...

[2022-09-14 21:34:13,482] adding document #4120000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:34:39,729] discarding 41857 tokens: [('waziro', 1), ('αβγανο', 1), ('παρσιῆται', 1), ('πασιανοί', 1), ('هډه', 1), ('ecosailingproject', 1), ('junlyu', 1), ('kleinmachnower', 1), ('aphelenchoide', 1), ('crusgali', 1)]...
[2022-09-14 21:34:39,731] keeping 2000000 tokens which were in no less than 0 and no more than 4130000 (=100.0%) documents
[2022-09-14 21:34:44,679] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:34:44,782] adding document #4130000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:35:10,684] discarding 35616 tokens: [('baarie', 1), ('baasit', 1), ('baasitt', 1), ('baattin', 1), ('baatwin', 1), ('baaʿeith', 1), ('baaʿith', 1), ('badiyʿ', 1), ('bariʿ', 1), ('basitt'

[2022-09-14 21:41:06,028] keeping 2000000 tokens which were in no less than 0 and no more than 4250000 (=100.0%) documents
[2022-09-14 21:41:10,068] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:41:10,171] adding document #4250000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:41:42,920] discarding 35438 tokens: [('cisarska', 1), ('dobřenských', 1), ('gammelnye', 1), ('jihoměstský', 1), ('juditin', 1), ('kolčavka', 1), ('malešický', 1), ('mikropivovar', 1), ('nosticovo', 1), ('pařížskou', 1)]...
[2022-09-14 21:41:42,923] keeping 2000000 tokens which were in no less than 0 and no more than 4260000 (=100.0%) documents
[2022-09-14 21:41:46,605] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:41:46,676] adding document #4260000 to Dictionary(2000000 unique t

[2022-09-14 21:47:59,709] discarding 32627 tokens: [('如果我沒有傷口', 1), ('最後的承諾', 1), ('畢國建', 1), ('阿冏', 1), ('sholze', 1), ('vezzoni', 1), ('realteam', 1), ('batubhai', 1), ('vasantvijay', 1), ('yuenanlu', 1)]...
[2022-09-14 21:47:59,712] keeping 2000000 tokens which were in no less than 0 and no more than 4380000 (=100.0%) documents
[2022-09-14 21:48:04,698] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:48:04,800] adding document #4380000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:48:34,757] discarding 32644 tokens: [('qirqisânî', 1), ('qozmím', 1), ('reinstalment', 1), ('retrodating', 1), ('rûfīn', 1), ('sambalut', 1), ('senaccherib', 1), ('shaprūṭ', 1), ('sâmânid', 1), ('tudrun', 1)]...
[2022-09-14 21:48:34,758] keeping 2000000 tokens which were in no less than 0 and no more than 4390000 (=100.0%) documents
[2022-09-14 21:48:39,766]

[2022-09-14 21:55:13,592] discarding 35029 tokens: [('kinderschutz', 1), ('arbaʽeen', 1), ('hasania', 1), ('khanqahe', 1), ('kichhauchha', 1), ('sasarami', 1), ('insurtec', 1), ('rightindem', 1), ('campillai', 1), ('funeral_mónica_echeverría_', 1)]...
[2022-09-14 21:55:13,593] keeping 2000000 tokens which were in no less than 0 and no more than 4510000 (=100.0%) documents
[2022-09-14 21:55:17,159] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:55:17,228] adding document #4510000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 21:55:42,253] discarding 29354 tokens: [('fmotq', 1), ('idnani', 1), ('anasis', 1), ('kmilsatcom', 1), ('ocaya', 1), ('twae', 1), ('yinwae', 1), ('pagenated', 1), ('chiofaro', 1), ('guidó', 1)]...
[2022-09-14 21:55:42,254] keeping 2000000 tokens which were in no less than 0 and no more than 4520000 (=100.0%) documents
[

[2022-09-14 22:02:29,658] discarding 30468 tokens: [('tanpiengco', 2), ('ametuer', 2), ('ghumurishi', 2), ('sagergaio', 2), ('seryogina', 2), ('kaushlesh', 2), ('ווארקא', 2), ('cuales', 2), ('riwsa', 2), ('gainlove', 2)]...
[2022-09-14 22:02:29,660] keeping 2000000 tokens which were in no less than 0 and no more than 4650000 (=100.0%) documents
[2022-09-14 22:02:33,270] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 22:02:33,339] adding document #4650000 to Dictionary(2000000 unique tokens: ['ability', 'ability#1', 'able', 'abolish', 'abolition']...)
[2022-09-14 22:02:59,860] discarding 32276 tokens: [('mimograptus', 2), ('tukuhnikivatz', 2), ('comedytheatrical', 2), ('distributorowned', 2), ('erfttal', 2), ('sevenpictures', 2), ('nbš', 2), ('olontigi', 2), ('babawi', 2), ('masakitnakatotohanan', 2)]...
[2022-09-14 22:02:59,862] keeping 2000000 tokens which were in no less than 0 and no more than 4660000 

In [6]:
for sent in wiki_sentences:
    print(sent[:100])
    break

['anarchism', 'political#1', 'philosophy', 'movement', 'sceptical', 'authority#4', 'reject', 'involuntary', 'coercive', 'form#11', 'hierarchy', 'anarchism', 'call#24', 'abolition', 'state', 'hold#15', 'unnecessary', 'undesirable', 'harmful', 'historically', 'left#7', 'wing', 'movement#3', 'placed', 'farthest', 'left', 'political', 'spectrum', 'usually', 'described', 'alongside', 'libertarian', 'marxism', 'libertarian', 'wing', 'libertarian', 'socialism', 'socialist', 'movement', 'ha', 'strong', 'historical#1', 'association', 'anti', 'capitalism', 'socialism', 'human', 'lived', 'society#1', 'without', 'formal#4', 'hierarchy#1', 'long#5', 'establishment#2', 'formal#3', 'state#3', 'realm', 'empire#1', 'rise#25', 'organised', 'hierarchical', 'body', 'scepticism', 'toward', 'authority#4', 'also', 'rose#18', 'th', 'century', 'self', 'conscious', 'political#1', 'movement#3', 'emerged', 'latter', 'half#1', 'th', 'first#11', 'decade', 'th', 'century', 'anarchist', 'movement', 'flourished', 'par

In [7]:
#sv.save(wiki_sentences,"wiki_sentences_pos_sample")
#sv.save(wiki_sentences,"wiki_sentences_pos")
#sv.save(wiki_sentences,"wiki_sentences_dep")
#sv.save(wiki_sentences,"wiki_sentences_sp")
#sv.save(wiki_sentences,"wiki_sentences_sp_loc")
#sv.save(wiki_sentences,"wiki_sentences_sp_ent")
#sv.save(wiki_sentences,"wiki_sentences_sp_ent_sample")
#sv.save(wiki_sentences,"wiki_sentences") # orignal
#sv.save(wiki_sentences,"wiki_sentences_dep2")
#sv.save(wiki_sentences,"wiki_sentences_uns")
#sv.save(wiki_sentences,"wiki_sentences_unsem")
#sv.save(wiki_sentences,"wiki_sentences_em")
sv.save(wiki_sentences,"wiki_sentences_wnet")

Saved the data


# Phrase mining

In [ ]:
from gensim.test.utils import datapath
from gensim.models.phrases import Phrases

In [ ]:
phrases = Phrases(sentences, min_count=100, threshold=1)
frozen_phrases = phrases.freeze()

In [ ]:
sv.save(phrases,"gensim_phrases")

# Train procedure

In [8]:
#sentences = sv.load("wiki_sentences_no")
#temp_sens are cased!!
#sentences = sv.load("temp_sens")
 
#sentences = sv.load("wiki_sentences") #Normal sentences using wiki_old.py

#Wiki_Sentences_SP are cased
#sentences = sv.load("Wiki_Sentences_SP")

#sentences = sv.load("wiki_sentences_sp_loc") #New
#sentences = sv.load("wiki_sentences_sp") #New

#sentences = sv.load("wiki_sentences_pos") # not to be used
#sentences = sv.load("Wiki_sentences_pos_sample")

#sentences = sv.load("wiki_sentences_sp_ent") # New
#sentences = sv.load("wiki_sentences_sp_ent_sample") # New

#sentences = sv.load("wiki_sentences_dep") #New
#sentences = sv.load("wiki_sentences_dep2") #New

#wiki english sample Cased 
#sentences = sv.load("Wiki_sentences_sp_sample")
#sentences = sv.load("wiki_sentences_uns") #New
#sentences = sv.load("wiki_sentences_unsem") #New
#sentences = sv.load("wiki_sentences_em") #New
sentences = sv.load("wiki_sentences_wnet") #New

In [9]:
#sentences = wiki_sentences
print("Minimum length of token:",sentences.wiki.token_min_len)

Minimum length of token: 1


In [ ]:
logging.info('Training model %s', 'wordnet')
model = word2vec.Word2Vec(sentences, window=10, sg=1, hs=0, negative=5, size=300, workers=40, iter=5)
logging.info('Training done.')

[2022-09-14 22:06:54,199] Training model wordnet
[2022-09-14 22:06:54,202] collecting all words and their counts
[2022-09-14 22:07:01,888] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2022-09-14 22:07:56,967] PROGRESS: at sentence #10000, processed 22225402 words, keeping 493671 word types
[2022-09-14 22:08:45,212] PROGRESS: at sentence #20000, processed 41943306 words, keeping 699785 word types
[2022-09-14 22:09:27,289] PROGRESS: at sentence #30000, processed 58290377 words, keeping 854217 word types
[2022-09-14 22:10:05,062] PROGRESS: at sentence #40000, processed 72364821 words, keeping 985354 word types
[2022-09-14 22:10:33,961] PROGRESS: at sentence #50000, processed 82317533 words, keeping 1065582 word types
[2022-09-14 22:10:50,722] PROGRESS: at sentence #60000, processed 87762646 words, keeping 1085962 word types
[2022-09-14 22:11:06,544] PROGRESS: at sentence #70000, processed 92716088 words, keeping 1104451 word types
[2022-09-14 22:11:20,963] PROGRESS: 

[2022-09-14 22:37:58,164] PROGRESS: at sentence #740000, processed 560241434 words, keeping 3991909 word types
[2022-09-14 22:38:17,680] PROGRESS: at sentence #750000, processed 565232033 words, keeping 4018322 word types
[2022-09-14 22:38:38,056] PROGRESS: at sentence #760000, processed 569992094 words, keeping 4041053 word types
[2022-09-14 22:38:56,126] PROGRESS: at sentence #770000, processed 574696074 words, keeping 4064701 word types
[2022-09-14 22:39:16,133] PROGRESS: at sentence #780000, processed 579515950 words, keeping 4091803 word types
[2022-09-14 22:39:39,714] PROGRESS: at sentence #790000, processed 584518584 words, keeping 4115549 word types
[2022-09-14 22:39:59,042] PROGRESS: at sentence #800000, processed 589205793 words, keeping 4138292 word types
[2022-09-14 22:40:18,902] PROGRESS: at sentence #810000, processed 593965467 words, keeping 4163275 word types
[2022-09-14 22:40:38,302] PROGRESS: at sentence #820000, processed 598640393 words, keeping 4186896 word types
[

[2022-09-14 23:01:47,382] PROGRESS: at sentence #1480000, processed 873553867 words, keeping 5539175 word types
[2022-09-14 23:02:05,917] PROGRESS: at sentence #1490000, processed 877320658 words, keeping 5555018 word types
[2022-09-14 23:02:25,443] PROGRESS: at sentence #1500000, processed 880903221 words, keeping 5573630 word types
[2022-09-14 23:02:44,791] PROGRESS: at sentence #1510000, processed 884553019 words, keeping 5590212 word types
[2022-09-14 23:03:02,748] PROGRESS: at sentence #1520000, processed 887829588 words, keeping 5604631 word types
[2022-09-14 23:03:19,299] PROGRESS: at sentence #1530000, processed 890984556 words, keeping 5616846 word types
[2022-09-14 23:03:39,887] PROGRESS: at sentence #1540000, processed 894572785 words, keeping 5632142 word types
[2022-09-14 23:03:59,326] PROGRESS: at sentence #1550000, processed 898200468 words, keeping 5651069 word types
[2022-09-14 23:04:17,822] PROGRESS: at sentence #1560000, processed 901869151 words, keeping 5670898 wor

[2022-09-14 23:26:26,650] PROGRESS: at sentence #2210000, processed 1136525524 words, keeping 6754740 word types
[2022-09-14 23:26:48,322] PROGRESS: at sentence #2220000, processed 1140118919 words, keeping 6771361 word types
[2022-09-14 23:27:17,768] PROGRESS: at sentence #2230000, processed 1143988367 words, keeping 6787536 word types
[2022-09-14 23:27:41,145] PROGRESS: at sentence #2240000, processed 1147606940 words, keeping 6803192 word types
[2022-09-14 23:27:59,975] PROGRESS: at sentence #2250000, processed 1150948558 words, keeping 6827112 word types
[2022-09-14 23:28:21,116] PROGRESS: at sentence #2260000, processed 1154576694 words, keeping 6843641 word types
[2022-09-14 23:28:41,197] PROGRESS: at sentence #2270000, processed 1158112213 words, keeping 6861814 word types
[2022-09-14 23:29:01,272] PROGRESS: at sentence #2280000, processed 1161618690 words, keeping 6877688 word types
[2022-09-14 23:29:21,622] PROGRESS: at sentence #2290000, processed 1165220484 words, keeping 68

[2022-09-14 23:52:24,665] PROGRESS: at sentence #2940000, processed 1395535912 words, keeping 7953384 word types
[2022-09-14 23:52:46,916] PROGRESS: at sentence #2950000, processed 1399068831 words, keeping 7970715 word types
[2022-09-14 23:53:08,069] PROGRESS: at sentence #2960000, processed 1402587199 words, keeping 7985385 word types
[2022-09-14 23:53:29,660] PROGRESS: at sentence #2970000, processed 1406168781 words, keeping 8001924 word types
[2022-09-14 23:53:51,934] PROGRESS: at sentence #2980000, processed 1409767963 words, keeping 8017041 word types
[2022-09-14 23:54:14,260] PROGRESS: at sentence #2990000, processed 1413254212 words, keeping 8028778 word types
[2022-09-14 23:54:38,413] PROGRESS: at sentence #3000000, processed 1416750382 words, keeping 8043647 word types
[2022-09-14 23:54:58,853] PROGRESS: at sentence #3010000, processed 1419970080 words, keeping 8057128 word types
[2022-09-14 23:55:20,826] PROGRESS: at sentence #3020000, processed 1423371147 words, keeping 80

[2022-09-15 00:19:42,909] PROGRESS: at sentence #3670000, processed 1647834352 words, keeping 8960541 word types
[2022-09-15 00:20:05,273] PROGRESS: at sentence #3680000, processed 1651205762 words, keeping 8971849 word types
[2022-09-15 00:20:26,362] PROGRESS: at sentence #3690000, processed 1654536101 words, keeping 8983907 word types
[2022-09-15 00:20:46,496] PROGRESS: at sentence #3700000, processed 1657794552 words, keeping 8995818 word types
[2022-09-15 00:21:08,105] PROGRESS: at sentence #3710000, processed 1661189683 words, keeping 9008698 word types
[2022-09-15 00:21:33,882] PROGRESS: at sentence #3720000, processed 1664767763 words, keeping 9021713 word types
[2022-09-15 00:21:57,241] PROGRESS: at sentence #3730000, processed 1668231750 words, keeping 9035295 word types
[2022-09-15 00:22:18,292] PROGRESS: at sentence #3740000, processed 1671590727 words, keeping 9048410 word types
[2022-09-15 00:22:38,068] PROGRESS: at sentence #3750000, processed 1674869710 words, keeping 90

[2022-09-15 00:47:43,013] PROGRESS: at sentence #4400000, processed 1887662242 words, keeping 9916099 word types
[2022-09-15 00:48:09,354] PROGRESS: at sentence #4410000, processed 1891376063 words, keeping 9967928 word types
[2022-09-15 00:48:36,559] PROGRESS: at sentence #4420000, processed 1894867412 words, keeping 9981518 word types
[2022-09-15 00:49:02,236] PROGRESS: at sentence #4430000, processed 1898177609 words, keeping 9993638 word types
[2022-09-15 00:49:29,055] PROGRESS: at sentence #4440000, processed 1901647529 words, keeping 10004658 word types
[2022-09-15 00:49:59,769] PROGRESS: at sentence #4450000, processed 1905380846 words, keeping 10016277 word types
[2022-09-15 00:50:24,033] PROGRESS: at sentence #4460000, processed 1908657844 words, keeping 10027745 word types
[2022-09-15 00:50:47,865] PROGRESS: at sentence #4470000, processed 1911868096 words, keeping 10040513 word types
[2022-09-15 00:51:09,114] PROGRESS: at sentence #4480000, processed 1915174304 words, keepin

[2022-09-15 02:39:54,565] EPOCHs No. 1 - PROGRESS: at 30.00% examples, 156019 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:41:58,633] EPOCHs No. 1 - PROGRESS: at 31.00% examples, 155714 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:44:06,063] EPOCHs No. 1 - PROGRESS: at 32.00% examples, 155280 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:46:04,274] EPOCHs No. 1 - PROGRESS: at 33.00% examples, 154872 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:48:12,855] EPOCHs No. 1 - PROGRESS: at 34.00% examples, 154386 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:50:17,400] EPOCHs No. 1 - PROGRESS: at 35.00% examples, 153997 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:52:27,978] EPOCHs No. 1 - PROGRESS: at 36.00% examples, 153414 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:54:39,067] EPOCHs No. 1 - PROGRESS: at 37.00% examples, 152833 words/s, in_qsize 0, out_qsize 0
[2022-09-15 02:57:07,602] EPOCHs No. 1 - PROGRESS: at 38.00% examples, 151955 words/s, in_qsize 0, out_qsize 0
[

[2022-09-15 05:17:22,201] worker thread finished; awaiting finish of 36 more threads
[2022-09-15 05:17:22,201] worker thread finished; awaiting finish of 35 more threads
[2022-09-15 05:17:22,202] worker thread finished; awaiting finish of 34 more threads
[2022-09-15 05:17:22,202] worker thread finished; awaiting finish of 33 more threads
[2022-09-15 05:17:22,204] worker thread finished; awaiting finish of 32 more threads
[2022-09-15 05:17:22,204] worker thread finished; awaiting finish of 31 more threads
[2022-09-15 05:17:22,205] worker thread finished; awaiting finish of 30 more threads
[2022-09-15 05:17:22,206] worker thread finished; awaiting finish of 29 more threads
[2022-09-15 05:17:22,206] worker thread finished; awaiting finish of 28 more threads
[2022-09-15 05:17:22,207] worker thread finished; awaiting finish of 27 more threads
[2022-09-15 05:17:22,207] worker thread finished; awaiting finish of 26 more threads
[2022-09-15 05:17:22,208] worker thread finished; awaiting finish

In [ ]:
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_filtered_sample.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_w2v_mc1_epoch5_300_sample.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2R_mc1_epoch5_300_filtered.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2w2v_mc1_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_con1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_neg10.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_neg10_w3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2S_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2B_mc100_epoch5_300_sub3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2TB_mc100_epoch5_300_LR.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2POS_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2DEP_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2LRM3_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2LOC_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx_mc100_epoch5_300_loc.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_w2v_mc100_epoch5_300_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_pos.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx_mc100_epoch5_300_ent_w10.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_dep2_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_ent_static_w3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_uns.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_unsem.txt'
#emb_file = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_uns_w1.txt'
emb_file = '/home/manni/embs/en_wiki_wnet_epoch5_300_w10.txt'

In [2]:
from gensim.models import KeyedVectors
wnet = '/home/manni/embs/en_wiki_wnet_epoch5_300_w10_new.txt'
model = KeyedVectors.load_word2vec_format(wnet, binary=False)

In [3]:
vocab = model.wv.vocab #dict

/tmp/ipykernel_31350/1054457911.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab = model.wv.vocab


In [4]:
len(vocab)

2702310

In [6]:
vocab.pop('[', None)
vocab.pop(']', None)
len(vocab)


2702310

In [9]:
_vocab = list(vocab.keys())
vocab = set()
for word in _vocab:
    if '§' in word:
        continue
    vocab.add(word)
len(vocab)

2614118

In [10]:
out_file = '/home/manni/embs/en_wiki_wnet_epoch5_300_w10_explicit.txt'

In [15]:
logging.info('Save trained word vectors')
with open(out_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(vocab), 300))
    for word in tqdm(vocab, position=0):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')

[2022-09-19 04:17:42,460] Save trained word vectors
  0%|                                                                                                                                                                     | 0/2614118 [00:00<?, ?it/s]/tmp/ipykernel_31350/2372546197.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2614118/2614118 [16:05<00:00, 2708.09it/s]
[2022-09-19 04:33:50,036] Done
